<a href="https://colab.research.google.com/github/fgsantosti/programacaodispositivosmoveis/blob/main/App_Flutter_06_ApiPokemon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Consumindo dados da PokeAPI**

#### 1. **Estrutura do Aplicativo**
O aplicativo é composto por duas telas principais:
- **Tela de Listagem**: Exibe uma lista com os nomes e imagens dos Pokémon. Cada item é clicável para abrir a tela de detalhes.
- **Tela de Detalhes**: Apresenta informações detalhadas sobre o Pokémon, como tipo, habilidades, altura e peso.

#### 2. **Biblioteca HTTP**
O pacote `http` é usado para consumir a API [PokeAPI](https://pokeapi.co/), que fornece os dados dos Pokémon.

#### 3. **Widgets Principais**
- `ListView.builder`: É usado na tela principal para renderizar dinamicamente a lista de Pokémon.
- `FutureBuilder`: Permite gerenciar o carregamento de dados e exibir estados como "carregando" ou "erro".

---

### **Organização em Arquivos**

Para manter o código modular, vamos dividir o aplicativo em três arquivos principais:

1. **`main.dart`**  
   Contém a configuração principal do aplicativo e define as rotas para navegação.

2. **`pokemon_list_page.dart`**  
   Contém o código da tela de listagem dos Pokémon, responsável por exibir os Pokémon com nome e imagem.

3. **`pokemon_detail_page.dart`**  
   Contém o código da tela de detalhes de um Pokémon específico.

---

### **Código Final Organizado**

#### **1. Arquivo `main.dart`**
```dart
import 'package:flutter/material.dart';
import 'pokemon_list_page.dart';

void main() {
  runApp(const PokedexApp());
}

class PokedexApp extends StatelessWidget {
  const PokedexApp({Key? key}) : super(key: key);

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Pokedex',
      theme: ThemeData(
        primarySwatch: Colors.red,
      ),
      home: const PokemonListPage(),
    );
  }
}
```

---

#### **2. Arquivo `pokemon_list_page.dart`**
```dart
import 'package:flutter/material.dart';
import 'package:http/http.dart' as http;
import 'dart:convert';
import 'pokemon_detail_page.dart';

class PokemonListPage extends StatefulWidget {
  const PokemonListPage({Key? key}) : super(key: key);

  @override
  _PokemonListPageState createState() => _PokemonListPageState();
}

class _PokemonListPageState extends State<PokemonListPage> {
  List<dynamic> _pokemonList = [];
  bool _isLoading = true;

  @override
  void initState() {
    super.initState();
    fetchPokemonData();
  }

  Future<void> fetchPokemonData() async {
    const apiUrl = 'https://pokeapi.co/api/v2/pokemon?limit=100';
    try {
      final response = await http.get(Uri.parse(apiUrl));
      if (response.statusCode == 200) {
        final data = json.decode(response.body);
        setState(() {
          _pokemonList = data['results'];
          _isLoading = false;
        });
      } else {
        throw Exception('Falha ao carregar os Pokémon');
      }
    } catch (error) {
      print(error);
      setState(() {
        _isLoading = false;
      });
    }
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Pokedex'),
      ),
      body: _isLoading
          ? const Center(child: CircularProgressIndicator())
          : ListView.builder(
              itemCount: _pokemonList.length,
              itemBuilder: (context, index) {
                final pokemon = _pokemonList[index];
                final pokemonId = index + 1;
                final imageUrl =
                    'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/$pokemonId.png';

                return ListTile(
                  leading: Image.network(imageUrl),
                  title: Text(pokemon['name']),
                  onTap: () {
                    Navigator.push(
                      context,
                      MaterialPageRoute(
                        builder: (context) =>
                            PokemonDetailPage(pokemonUrl: pokemon['url']),
                      ),
                    );
                  },
                );
              },
            ),
    );
  }
}
```

---

#### **3. Arquivo `pokemon_detail_page.dart`**
```dart
import 'package:flutter/material.dart';
import 'package:http/http.dart' as http;
import 'dart:convert';

class PokemonDetailPage extends StatefulWidget {
  final String pokemonUrl;

  const PokemonDetailPage({Key? key, required this.pokemonUrl})
      : super(key: key);

  @override
  _PokemonDetailPageState createState() => _PokemonDetailPageState();
}

class _PokemonDetailPageState extends State<PokemonDetailPage> {
  Map<String, dynamic>? _pokemonDetails;
  bool _isLoading = true;

  @override
  void initState() {
    super.initState();
    fetchPokemonDetails();
  }

  Future<void> fetchPokemonDetails() async {
    try {
      final response = await http.get(Uri.parse(widget.pokemonUrl));
      if (response.statusCode == 200) {
        final data = json.decode(response.body);
        setState(() {
          _pokemonDetails = data;
          _isLoading = false;
        });
      } else {
        throw Exception('Falha ao carregar os detalhes do Pokémon');
      }
    } catch (error) {
      print(error);
      setState(() {
        _isLoading = false;
      });
    }
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Detalhes do Pokémon'),
      ),
      body: _isLoading
          ? const Center(child: CircularProgressIndicator())
          : _pokemonDetails == null
              ? const Center(child: Text('Erro ao carregar os dados'))
              : Padding(
                  padding: const EdgeInsets.all(16.0),
                  child: Column(
                    crossAxisAlignment: CrossAxisAlignment.start,
                    children: [
                      Center(
                        child: Image.network(
                          _pokemonDetails!['sprites']['front_default'],
                          height: 150,
                          width: 150,
                        ),
                      ),
                      const SizedBox(height: 16),
                      Text(
                        'Nome: ${_pokemonDetails!['name']}',
                        style: const TextStyle(
                            fontSize: 20, fontWeight: FontWeight.bold),
                      ),
                      const SizedBox(height: 8),
                      Text(
                        'Altura: ${_pokemonDetails!['height']} decímetros',
                      ),
                      Text(
                        'Peso: ${_pokemonDetails!['weight']} hectogramas',
                      ),
                      const SizedBox(height: 8),
                      Text(
                        'Tipos:',
                        style: const TextStyle(fontWeight: FontWeight.bold),
                      ),
                      Column(
                        crossAxisAlignment: CrossAxisAlignment.start,
                        children: _pokemonDetails!['types']
                            .map<Widget>((type) => Text(
                                  '- ${type['type']['name']}',
                                ))
                            .toList(),
                      ),
                      const SizedBox(height: 8),
                      Text(
                        'Habilidades:',
                        style: const TextStyle(fontWeight: FontWeight.bold),
                      ),
                      Column(
                        crossAxisAlignment: CrossAxisAlignment.start,
                        children: _pokemonDetails!['abilities']
                            .map<Widget>((ability) => Text(
                                  '- ${ability['ability']['name']}',
                                ))
                            .toList(),
                      ),
                    ],
                  ),
                ),
    );
  }
}
```

---

### **Estrutura de Arquivos**
```plaintext
lib/
├── main.dart
├── pokemon_list_page.dart
└── pokemon_detail_page.dart
```

### **Passos para Testar**
1. **Clone ou crie um novo projeto Flutter**.  
2. Adicione os arquivos no diretório `lib`.  
3. Adicione a dependência `http` no arquivo `pubspec.yaml` e execute `flutter pub get`.  
4. Execute o aplicativo no simulador ou dispositivo real.  

Essa organização modular facilita futuras manutenções e expansões do aplicativo. 🚀